### Libraries


In [2]:
# %pip install optuna

In [3]:
import os
from pathlib import Path
import gc
import numpy as np
import pandas as pd

import lightgbm as lgb
import optuna

### Config


In [ ]:
SEED = 42
np.random.seed(SEED)

HORIZON = 16
MAX_LAG = 28

# Tuning setup
TUNE_START_DATE = "2016-01-01"  # speed things up (use recent history only)
N_FOLDS_TUNE = 3
N_TRIALS = 25
N_ESTIMATORS = 2000

# Paths
DATA_DIR = Path("/content/data")

# Files
TRAIN_PATH = DATA_DIR / "train.csv"
TEST_PATH = DATA_DIR / "test.csv"
STORES_PATH = DATA_DIR / "stores.csv"
OIL_PATH = DATA_DIR / "oil.csv"
HOLIDAYS_PATH = DATA_DIR / "holidays_events.csv"
SAMPLE_SUB_PATH = DATA_DIR / "sample_submission.csv"

### Import Data


In [5]:
train_dtypes = {
    "id": "int32",
    "store_nbr": "int16",
    "family": "string",
    "sales": "float32",
    "onpromotion": "int16",
}
test_dtypes = {
    "id": "int32",
    "store_nbr": "int16",
    "family": "string",
    "onpromotion": "int16",
}
stores_dtypes = {
    "store_nbr": "int16",
    "city": "string",
    "state": "string",
    "type": "string",
    "cluster": "int16",
}

train = pd.read_csv(TRAIN_PATH, dtype=train_dtypes, parse_dates=["date"])
test = pd.read_csv(TEST_PATH, dtype=test_dtypes, parse_dates=["date"])
stores = pd.read_csv(STORES_PATH, dtype=stores_dtypes)
oil = pd.read_csv(OIL_PATH, parse_dates=["date"])
holidays = pd.read_csv(HOLIDAYS_PATH, parse_dates=["date"])

print(train.shape, test.shape, stores.shape, oil.shape, holidays.shape)

train_min_date, train_max_date = train["date"].min(), train["date"].max()
test_min_date, test_max_date = test["date"].min(), test["date"].max()
print("Train:", train_min_date.date(), "->", train_max_date.date())
print("Test :", test_min_date.date(), "->", test_max_date.date())

(3000888, 6) (28512, 5) (54, 5) (1218, 2) (350, 6)
Train: 2013-01-01 -> 2017-08-15
Test : 2017-08-16 -> 2017-08-31


### Data Cleaning & Preprocessing


In [ ]:
def rmsle(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=np.float64)
    y_pred = np.asarray(y_pred, dtype=np.float64)
    y_pred = np.maximum(y_pred, 0.0)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))


def add_date_features(df):
    d = df["date"]
    df["year"] = d.dt.year.astype("int16")
    df["month"] = d.dt.month.astype("int8")
    df["day"] = d.dt.day.astype("int8")
    df["dow"] = d.dt.dayofweek.astype("int8")
    df["week"] = d.dt.isocalendar().week.astype("int16")
    df["doy"] = d.dt.dayofyear.astype("int16")
    df["is_weekend"] = (df["dow"] >= 5).astype("int8")
    df["is_month_start"] = d.dt.is_month_start.astype("int8")
    df["is_month_end"] = d.dt.is_month_end.astype("int8")

    # Cyclic encodings
    df["dow_sin"] = np.sin(2 * np.pi * df["dow"] / 7).astype("float32")
    df["dow_cos"] = np.cos(2 * np.pi * df["dow"] / 7).astype("float32")
    df["doy_sin"] = np.sin(2 * np.pi * df["doy"] / 365.25).astype("float32")
    df["doy_cos"] = np.cos(2 * np.pi * df["doy"] / 365.25).astype("float32")
    return df


# Oil: forward/back fill
oil = oil.sort_values("date")
oil["dcoilwtico"] = oil["dcoilwtico"].astype("float32").ffill().bfill()

# Holidays: keep non-transferred; create national/regional/local counts by type
hol = holidays.copy()
hol = hol[hol["transferred"] == False].copy()

for col in ["type", "locale", "locale_name"]:
    hol[col] = hol[col].astype("string")


def holiday_counts(df, group_cols, prefix):
    tmp = df.copy()
    tmp["is_holiday"] = (tmp["type"] == "Holiday").astype("int16")
    tmp["is_event"] = (tmp["type"] == "Event").astype("int16")
    tmp["is_additional"] = (tmp["type"] == "Additional").astype("int16")
    tmp["is_bridge"] = (tmp["type"] == "Bridge").astype("int16")
    tmp["is_workday"] = (tmp["type"] == "Work Day").astype("int16")
    agg = (
        tmp.groupby(group_cols)[
            ["is_holiday", "is_event", "is_additional", "is_bridge", "is_workday"]
        ]
        .sum()
        .reset_index()
    )
    agg = agg.rename(
        columns={
            "is_holiday": f"{prefix}_holiday",
            "is_event": f"{prefix}_event",
            "is_additional": f"{prefix}_additional",
            "is_bridge": f"{prefix}_bridge",
            "is_workday": f"{prefix}_workday",
        }
    )
    return agg


hol_nat = holiday_counts(hol[hol["locale"] == "National"], ["date"], "nat")
hol_reg = holiday_counts(
    hol[hol["locale"] == "Regional"].rename(columns={"locale_name": "state"}),
    ["date", "state"],
    "reg",
)
hol_loc = holiday_counts(
    hol[hol["locale"] == "Local"].rename(columns={"locale_name": "city"}),
    ["date", "city"],
    "loc",
)

# Categoricals
for c in ["city", "state", "type"]:
    stores[c] = stores[c].astype("category")
stores["cluster"] = stores["cluster"].astype("int16").astype("category")

### EDA


In [7]:
print(
    "Unique stores:",
    train["store_nbr"].nunique(),
    "Unique families:",
    train["family"].nunique(),
)
print("Sales zeros %:", (train["sales"] == 0).mean())

Unique stores: 54 Unique families: 33
Sales zeros %: 0.3129506999261552


### Feature Engineering & Selection


In [ ]:
# Build base (train+test)
base = pd.concat([train.drop(columns=["sales"]), test], ignore_index=True)

# Categoricals
base["family"] = base["family"].astype("category")
base["store_nbr"] = base["store_nbr"].astype("int16").astype("category")

base = base.merge(stores, on="store_nbr", how="left")
base = base.merge(oil, on="date", how="left")
base = base.merge(hol_nat, on="date", how="left")
base = base.merge(hol_reg, on=["date", "state"], how="left")
base = base.merge(hol_loc, on=["date", "city"], how="left")

# Fill holiday missing counts
holiday_cols = [c for c in base.columns if c.startswith(("nat_", "reg_", "loc_"))]
for c in holiday_cols:
    base[c] = base[c].fillna(0).astype("int16")

# Fill oil missing
base["dcoilwtico"] = (
    base["dcoilwtico"].fillna(base["dcoilwtico"].median()).astype("float32")
)

base = add_date_features(base)

base["family"] = base["family"].astype("category")
base["store_nbr"] = base["store_nbr"].astype("int16").astype("category")

# Series id for efficient grouping
# Use stable integer codes for family + store
base["family_code"] = base["family"].cat.codes.astype("int16")
base["store_code"] = base["store_nbr"].cat.codes.astype("int16")
base["series_id"] = (
    base["store_code"].astype("int32") * 100 + base["family_code"].astype("int32")
).astype("int32")

# Promotion lags/rolling
base = base.sort_values(["series_id", "date"])
g_promo = base.groupby("series_id")["onpromotion"]

for lag in [1, 7, 14, 28]:
    base[f"promo_lag_{lag}"] = g_promo.shift(lag).astype("float32")

shifted_promo = g_promo.shift(1)
for w in [7, 28]:
    base[f"promo_rollmean_{w}"] = (
        shifted_promo.rolling(w)
        .mean()
        .reset_index(level=0, drop=True)
        .astype("float32")
    )

promo_cols = [c for c in base.columns if c.startswith("promo_")]
for c in promo_cols:
    base[c] = base[c].fillna(0.0).astype("float32")

# Build train feature table with sales lags
train_feat = train.merge(
    base.drop(columns=["family_code", "store_code"]),  # keep series_id etc
    on=["id", "date", "store_nbr", "family", "onpromotion"],
    how="left",
)

train_feat["log_sales"] = np.log1p(train_feat["sales"].astype("float32")).astype(
    "float32"
)

train_feat = train_feat.sort_values(["series_id", "date"])
g_sales = train_feat.groupby("series_id")["log_sales"]

for lag in [1, 7, 14, 28]:
    train_feat[f"sales_lag_{lag}"] = g_sales.shift(lag).astype("float32")

shifted_sales = g_sales.shift(1)
for w in [7, 28]:
    train_feat[f"sales_rollmean_{w}"] = (
        shifted_sales.rolling(w)
        .mean()
        .reset_index(level=0, drop=True)
        .astype("float32")
    )

sales_lag_cols = [f"sales_lag_{x}" for x in [1, 7, 14, 28]] + [
    f"sales_rollmean_{x}" for x in [7, 28]
]
for c in sales_lag_cols:
    train_feat[c] = train_feat[c].fillna(0.0).astype("float32")

# Final categorical casting
cat_cols = ["store_nbr", "family", "city", "state", "type", "cluster"]
for c in cat_cols:
    train_feat[c] = train_feat[c].astype("category")
    base[c] = base[c].astype("category")

# Feature list
date_cols = [
    "year",
    "month",
    "day",
    "dow",
    "week",
    "doy",
    "is_weekend",
    "is_month_start",
    "is_month_end",
    "dow_sin",
    "dow_cos",
    "doy_sin",
    "doy_cos",
]
core_cols = [
    "store_nbr",
    "family",
    "city",
    "state",
    "type",
    "cluster",
    "onpromotion",
    "dcoilwtico",
] + holiday_cols
feature_cols = core_cols + date_cols + promo_cols + sales_lag_cols

# clean things up
train_feat = train_feat.dropna(subset=feature_cols + ["log_sales"]).reset_index(
    drop=True
)

print("Train features:", train_feat.shape, "Base:", base.shape)

Train features: (3000888, 53) Base: (3029400, 47)


### Model Training (LightGBM)


In [ ]:
def make_folds(max_date, horizon=16, n_folds=3):
    # rolling origin with contiguous blocks
    folds = []
    for i in range(n_folds):
        val_end = max_date - pd.Timedelta(days=horizon * i)
        val_start = val_end - pd.Timedelta(days=horizon - 1)
        train_end = val_start - pd.Timedelta(days=1)
        folds.append((train_end, val_start, val_end))
    return folds[::-1]  # oldest -> newest


def recursive_predict(
    model, base_df, hist_df, start_date, end_date, feature_cols, max_lag=28
):
    # base_df must contain all rows for the prediction dates.
    dates = pd.date_range(start_date, end_date, freq="D")

    df0 = base_df[base_df["date"] == dates[0]].sort_values("series_id")
    series_ids = df0["series_id"].to_numpy()
    n = len(series_ids)

    init_end = dates[0] - pd.Timedelta(days=1)
    hist = np.zeros((n, max_lag), dtype=np.float32)

    tmp = (
        hist_df[hist_df["date"] <= init_end]
        .sort_values(["series_id", "date"])
        .groupby("series_id")["log_sales"]
        .apply(lambda s: s.to_numpy()[-max_lag:])
    )

    # align buffers
    for i, sid in enumerate(series_ids):
        arr = tmp.get(sid)
        if arr is None:
            continue
        if len(arr) < max_lag:
            hist[i, -len(arr) :] = arr
        else:
            hist[i, :] = arr

    out = []
    for d in dates:
        df_day = base_df[base_df["date"] == d].sort_values("series_id").copy()

        # sales lags from buffer
        df_day["sales_lag_1"] = hist[:, -1]
        df_day["sales_lag_7"] = hist[:, -7]
        df_day["sales_lag_14"] = hist[:, -14]
        df_day["sales_lag_28"] = hist[:, -28]
        df_day["sales_rollmean_7"] = hist[:, -7:].mean(axis=1)
        df_day["sales_rollmean_28"] = hist[:, -28:].mean(axis=1)

        X = df_day[feature_cols]
        pred_log = model.predict(X)
        pred = np.maximum(np.expm1(pred_log), 0.0)

        out.append(pd.DataFrame({"id": df_day["id"].values, "sales": pred}))

        # update buffer with predictions
        hist = np.roll(hist, -1, axis=1)
        hist[:, -1] = pred_log.astype(np.float32)

    return pd.concat(out, ignore_index=True)


# Data for history lookup during recursion (full train sales history)
hist_df_full = train_feat[["date", "series_id", "log_sales"]].copy()

### Cross Validation & Hyperparameter Tuning (Optuna)


In [ ]:
max_date = train_feat["date"].max()
folds = make_folds(max_date, horizon=HORIZON, n_folds=N_FOLDS_TUNE)

# To speed things up: tune on recent slice only (training rows), but keep full history for recursion
tune_mask = train_feat["date"] >= pd.to_datetime(TUNE_START_DATE)
train_tune = train_feat.loc[tune_mask].reset_index(drop=True)

# Base rows needed for recursive validation dates
base_small = base.copy()


def objective(trial):
    params = {
        "n_estimators": N_ESTIMATORS,
        "learning_rate": trial.suggest_float("learning_rate", 0.03, 0.15, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 200),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 10.0),
        "max_depth": trial.suggest_int("max_depth", 6, 14),
        "objective": "regression",
        "random_state": SEED,
        "n_jobs": -1,
        "verbosity": -1,
        "force_row_wise": True,
    }

    scores = []
    for fold_idx, (train_end, val_start, val_end) in enumerate(folds):
        # train rows restricted to <= train_end and within tuning slice
        tr = train_tune[train_tune["date"] <= train_end]
        if tr.empty:
            return np.inf

        X_tr = tr[feature_cols]
        y_tr = tr["log_sales"]

        model = lgb.LGBMRegressor(**params)
        model.fit(X_tr, y_tr, categorical_feature=cat_cols)

        pred_val = recursive_predict(
            model=model,
            base_df=base_small,
            hist_df=hist_df_full[hist_df_full["date"] <= train_end],
            start_date=val_start,
            end_date=val_end,
            feature_cols=feature_cols,
            max_lag=MAX_LAG,
        )

        truth = train.loc[
            (train["date"] >= val_start) & (train["date"] <= val_end), ["id", "sales"]
        ]
        merged = truth.merge(
            pred_val, on="id", how="inner", suffixes=("_true", "_pred")
        )
        score = rmsle(merged["sales_true"], merged["sales_pred"])
        scores.append(score)

        trial.report(float(np.mean(scores)), step=fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

        del model, tr, X_tr, y_tr, pred_val, truth, merged
        gc.collect()

    return float(np.mean(scores))


study = optuna.create_study(
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=0),
)
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

print("Best score:", study.best_value)
print("Best params:", study.best_params)

[I 2026-02-04 15:38:07,297] A new study created in memory with name: no-name-bd239d3b-444d-42d6-a229-a6d5ee3bea06


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2026-02-04 15:43:28,113] Trial 0 finished with value: 0.40153553839667727 and parameters: {'learning_rate': 0.054816785328198704, 'num_leaves': 244, 'min_child_samples': 152, 'feature_fraction': 0.8394633936788146, 'bagging_fraction': 0.6624074561769746, 'bagging_freq': 2, 'lambda_l1': 0.2904180608409973, 'lambda_l2': 8.661761457749352, 'max_depth': 11}. Best is trial 0 with value: 0.40153553839667727.
[I 2026-02-04 15:46:41,142] Trial 1 finished with value: 0.40400603833570603 and parameters: {'learning_rate': 0.09376542954502831, 'num_leaves': 35, 'min_child_samples': 195, 'feature_fraction': 0.9329770563201687, 'bagging_fraction': 0.6849356442713105, 'bagging_freq': 2, 'lambda_l1': 0.9170225492671691, 'lambda_l2': 3.0424224295953772, 'max_depth': 10}. Best is trial 0 with value: 0.40153553839667727.
[I 2026-02-04 15:50:39,210] Trial 2 finished with value: 0.3993831102499141 and parameters: {'learning_rate': 0.06012261562962467, 'num_leaves': 96, 'min_child_samples': 130, 'feature

### Testing & Evaluation


In [11]:
best_params = study.best_params.copy()
final_params = {
    "n_estimators": N_ESTIMATORS,
    "objective": "regression",
    "random_state": SEED,
    "n_jobs": -1,
    "verbosity": -1,
    "force_row_wise": True,
    **best_params,
}

final_model = lgb.LGBMRegressor(**final_params)
final_model.fit(
    train_feat[feature_cols], train_feat["log_sales"], categorical_feature=cat_cols
)

# Predict test recursively
pred_test = recursive_predict(
    model=final_model,
    base_df=base,
    hist_df=hist_df_full,
    start_date=test_min_date,
    end_date=test_max_date,
    feature_cols=feature_cols,
    max_lag=MAX_LAG,
)

sub = pd.read_csv(SAMPLE_SUB_PATH)
sub = sub.drop(columns=["sales"], errors="ignore").merge(pred_test, on="id", how="left")
sub["sales"] = sub["sales"].fillna(0.0).astype("float32")

sub_path = "submission.csv"
sub.to_csv(sub_path, index=False)
print("Saved:", sub_path, "shape:", sub.shape)
sub.head()

Saved: submission.csv shape: (28512, 2)


,id,sales
0,3000888,4.200418
1,3000889,0.040689
2,3000890,6.043622
3,3000891,2410.119141
4,3000892,0.041520
